In [ ]:
import numpy as np
import pandas as pd

class BSDICalculator:
    def __init__(self, data, indicators, invert_flags=None):
        """
        Parameters:
        - data: pd.DataFrame containing the variables.
        - indicators: list of column names to be used for BSDI (e.g., ['RGDP', 'EnvHealth', 'Population', 'Groundwater']).
        - invert_flags: list of booleans indicating which indicators should be inversely normalized (default: False for all).
        """
        self.data = data.copy()
        self.indicators = indicators
        self.invert_flags = invert_flags if invert_flags else [False] * len(indicators)
        self.normalized_data = pd.DataFrame()

    def normalize_column(self, column, invert=False):
        series = self.data[column].astype(float)
        v_min = series.min()
        v_max = series.max()
        if v_max == v_min:
            return pd.Series([1.0] * len(series), index=series.index)
        norm = (v_max - series) / (v_max - v_min) if invert else (series - v_min) / (v_max - v_min)
        return norm.clip(0, 1)

    def compute_normalized_indicators(self):
        for col, inv in zip(self.indicators, self.invert_flags):
            self.normalized_data[col] = self.normalize_column(col, invert=inv)

    def compute_bsdi(self):
        self.compute_normalized_indicators()
        product = self.normalized_data.prod(axis=1)
        bsdi = np.sqrt(product)
        return bsdi

    def run(self):
        bsdi = self.compute_bsdi()
        result_df = self.data.copy()
        result_df['BSDI'] = bsdi
        return result_df


